In [1]:
%pip install requests

Note: you may need to restart the kernel to use updated packages.


In [2]:
teams = ["TOR"] # Will include all teams later on

In [3]:
url_format = "https://www.basketball-reference.com/teams/{}/2025.html"

In [4]:
%pip install beautifulsoup4

Note: you may need to restart the kernel to use updated packages.


In [5]:
!python -m pip install certifi

In [ ]:
#to make sure we aren't bot
import certifi
import urllib3
http = urllib3.PoolManager(
    cert_reqs='CERT_REQUIRED',
    ca_certs=certifi.where()
)

In [7]:
import requests
from bs4 import BeautifulSoup
import time
import os

# If you get an error, try printing response. If it returns <Reponse [429]>, you are sending too many requests, you have to wait a while to run again
# Check the reponse header to see how long you have to wait before running the req again (we can implement this automatically (optional)), usually 1 hour
# This function will take a while to prevent overflowing the limit (max 30 req /min)

# Since this takes so long, one team might need to be updated on seperate days / and maybe every week worst case

# We use certification to adhere with best practices (this doesn't work on corporate wifi)
# Right now, it sets verification to false, uncomment and replace this line with the one below to use certification

# Initial runtime for the Raptors took 4 min 27.2 seconds 

for team in teams:
    url = url_format.format(team)
    time.sleep(2)
    #response = requests.get(url, verify=False)
    response = requests.get(url, verify=certifi.where())
    print(response)
    print(response.headers)
    soup = BeautifulSoup(response.content, 'html.parser')

    roster =  soup.find("table", {"id": "roster"})

    for row in roster.find_all("tr"):
        player_cells = row.find_all('td', {'data-stat': 'player'})

        for cell in player_cells:
            link_tag = cell.find('a')
            if link_tag:
                player_link = link_tag['href']

                full_player_link_format = "https://www.basketball-reference.com{}"
                full_player_link = full_player_link_format.format(player_link)
                time.sleep(2)
                #player_response = requests.get(full_player_link, verify=False)
                player_response = requests.get(full_player_link, verify=certifi.where())
                player_soup = BeautifulSoup(player_response.content, 'html.parser')

                game_logs_header = player_soup.find('p', {'class': 'listhead'}, string="Game Logs")
                    
                if game_logs_header:
                    game_log_links = game_logs_header.find_next_sibling('ul')

                    if game_log_links:
                        for link in game_log_links.find_all('a', href=True):
                            nested_link = link['href']
                            print(f"Nested link: https://www.basketball-reference.com{nested_link}")

                            player_name = player_link.split('/')[-1].replace('.html', '')
                            game_log_link_format = "https://www.basketball-reference.com{}"
                            game_log_link = game_log_link_format.format(nested_link)
                            time.sleep(2)
                            game_log_response = requests.get(game_log_link, verify=certifi.where())

                            year = game_log_link.split('/')[-1]

                            if year == "":
                                year = "playoffs"

                            # Create the directory if it doesn't exist
                            directory = f"data/{team}/{player_name}"
                            if not os.path.exists(directory):
                                os.makedirs(directory)

                            # Replace this with a request to a relational database (MongoDB)
                            with open(f"{directory}/{year}.html", 'a', encoding='utf-8') as file:
                                file.write(game_log_response.text)

<Response [200]>
{'Date': 'Thu, 07 Nov 2024 01:47:31 GMT', 'Content-Type': 'text/html', 'Transfer-Encoding': 'chunked', 'Connection': 'keep-alive', 'strict-transport-security': 'max-age=2592000; includeSubDomains', 'vary': 'Accept-Encoding', 'Last-Modified': 'Thu, 07 Nov 2024 01:32:29 GMT', 'CF-Cache-Status': 'HIT', 'Set-Cookie': '__cf_bm=nx8cnUaikYqLzA6JI89I.2M3t57kHy49fdXeGEEo.I4-1730944051-1.0.1.1-3nxy074T.TlsX8IdUvY6I_UuFoB458oq6_xaBB_CoEL6fgp.Z0GforOYLRaeEIknMTQvM5e9sDTiYKgF5BOz4Q; path=/; expires=Thu, 07-Nov-24 02:17:31 GMT; domain=.basketball-reference.com; HttpOnly; Secure; SameSite=None', 'Server': 'cloudflare', 'CF-RAY': '8de9a7e11d3fac18-YYZ', 'Content-Encoding': 'gzip'}
Nested link: https://www.basketball-reference.com/players/d/dickgr01/gamelog/2024
Nested link: https://www.basketball-reference.com/players/d/dickgr01/gamelog/2025
Nested link: https://www.basketball-reference.com/players/p/poeltja01/gamelog/2017
Nested link: https://www.basketball-reference.com/players/p/po

In [8]:
%pip install pandas
%pip install pyarrow

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [9]:
%pip install bs4
%pip install html5lib
%pip install lxml

Note: you may need to restart the kernel to use updated packages.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 112.2/112.2 kB 2.7 MB/s eta 0:00:00a 0:00:01
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [10]:
import pandas as pd

In [11]:
import os
import re
from io import StringIO
import pandas as pd
from bs4 import BeautifulSoup

# Does not work for playoff files yet (different table format)

dfs = []

outer_folder = 'data'

pattern = r"^(.*?)(?=\s\d{4}-\d{2}|\splayoffs)"

for dirpath, dirnames, filenames in os.walk(outer_folder):
    for filename in filenames:

        file_path = os.path.join(dirpath, filename)
        
        with open(file_path, 'r', encoding='utf-8') as file:
            page = file.read()

        soup = BeautifulSoup(page, 'html.parser')

        # Remove middle headers
        middle_headers = soup.find_all('tr', class_="thead")

        for header in middle_headers:
            header.decompose()

        game_log_table = soup.find('table', {'id': 'pgl_basic'})

        print("File path: ", file_path)

        if game_log_table is None:
            print(f"No table with ID 'pgl_basic' found in file {file_path}")
            continue  # Skip this file and move to the next

        title_tag = soup.find('title')

        if title_tag:
            title_text = title_tag.get_text()
            match = re.search(pattern, title_text)

            if match:
                full_name = match.group(1).strip() 

        game_log = pd.read_html(StringIO(str(game_log_table)))[0]
       
        if full_name:
            game_log["Name"] = full_name

            # Reorder the columns to put "Name" first
            columns = ['Name'] + [col for col in game_log.columns if col != 'Name']
            game_log = game_log[columns]

        dfs.append(game_log)

File path:  data/TOR/brownbr01/2021.html
File path:  data/TOR/brownbr01/2020.html
File path:  data/TOR/brownbr01/2024.html
File path:  data/TOR/brownbr01/playoffs.html
No table with ID 'pgl_basic' found in file data/TOR/brownbr01/playoffs.html
File path:  data/TOR/brownbr01/2023.html
File path:  data/TOR/brownbr01/2019.html
File path:  data/TOR/brownbr01/2022.html
File path:  data/TOR/bouchch01/2021.html
File path:  data/TOR/bouchch01/2020.html
File path:  data/TOR/bouchch01/2025.html
File path:  data/TOR/bouchch01/2024.html
File path:  data/TOR/bouchch01/playoffs.html
No table with ID 'pgl_basic' found in file data/TOR/bouchch01/playoffs.html
File path:  data/TOR/bouchch01/2023.html
File path:  data/TOR/bouchch01/2019.html
File path:  data/TOR/bouchch01/2018.html
File path:  data/TOR/bouchch01/2022.html
File path:  data/TOR/mitchda01/2025.html
File path:  data/TOR/mitchda01/2024.html
File path:  data/TOR/mitchda01/playoffs.html
No table with ID 'pgl_basic' found in file data/TOR/mitch

In [21]:
game_logs = pd.concat(dfs)
game_logs.drop(['Unnamed: 5', "Rk", "G"], axis=1, inplace=True)
game_logs.rename(columns={'Unnamed: 7': 'Win/Loss'}, inplace=True)

In [ ]:
team_names = {
    'ATL': 'Atlanta Hawks', 
    'BOS': 'Boston Celtics',
    'BKN': 'Brooklyn Nets',
    'CHO': 'Charlotte Hornets',
    'CHI': 'Chicago Bulls',
    'CLE': 'Cleveland Cavaliers',
    'DAL': 'Dallas Mavericks',
    'DEN': 'Denver Nuggets',
    'DET': 'Detroit Pistons',
    'GSW': 'Golden State Warriors',
    'HOU': 'Houston Rockets',
    'IND': 'Indiana Pacers',
    'LAC': 'Los Angeles Clippers',
    'LAL': 'Los Angeles Lakers',
    'MEM': 'Memphis Grizzlies',
    'MIA': 'Miami Heat',
    'MIL': 'Milwaukee Bucks',
    'MIN': 'Minnesota Timberwolves',
    'NOP': 'New Orleans Pelicans',
    'NYK': 'New York Knicks',
    'OKC': 'Oklahoma City Thunder',
    'ORL': 'Orlando Magic',
    'PHI': 'Philadelphia 76ers',
    'PHX': 'Phoenix Suns',
    'POR': 'Portland Trail Blazers',
    'SAC': 'Sacramento Kings',
    'SAS': 'San Antonio Spurs',
    'TOR': 'Toronto Raptors',
    'UTA': 'Utah Jazz',
    'WAS': 'Washington Wizards'
}

In [22]:
# Have to replace opponent names with full names to merge dataframes in the future based on a common field (opponent)
game_logs['Opp'] = game_logs['Opp'].map(team_names)

In [23]:
game_logs.to_csv("game_logs.csv")

In [24]:
game_logs.head(60)

,Name,Date,Age,Tm,Opp,Win/Loss,GS,MP,FG,FGA,...,DRB,TRB,AST,STL,BLK,TOV,PF,PTS,GmSc,+/-
0,Bruce Brown,2020-12-22,24-129,BRK,Golden State Warriors,W (+26),0,7:31,1,3,...,1,1,1,0,0,2,1,2,-1.1,-7
1,Bruce Brown,2020-12-25,24-132,BRK,Boston Celtics,W (+28),0,2:30,0,0,...,1,1,0,0,0,0,0,0,0.3,+4
2,Bruce Brown,2020-12-27,24-134,BRK,Charlotte Hornets,L (-2),Did Not Play,Did Not Play,Did Not Play,Did Not Play,...,Did Not Play,Did Not Play,Did Not Play,Did Not Play,Did Not Play,Did Not Play,Did Not Play,Did Not Play,Did Not Play,Did Not Play
3,Bruce Brown,2020-12-28,24-135,BRK,Memphis Grizzlies,L (-5),Did Not Play,Did Not Play,Did Not Play,Did Not Play,...,Did Not Play,Did Not Play,Did Not Play,Did Not Play,Did Not Play,Did Not Play,Did Not Play,Did Not Play,Did Not Play,Did Not Play
4,Bruce Brown,2020-12-30,24-137,BRK,Atlanta Hawks,W (+4),Did Not Play,Did Not Play,Did Not Play,Did Not Play,...,Did Not Play,Did Not Play,Did Not Play,Did Not Play,Did Not Play,Did Not Play,Did Not Play,Did Not Play,Did Not Play,Did Not Play
5,Bruce Brown,2021-01-01,24-139,BRK,Atlanta Hawks,L (-18),0,3:00,1,1,...,0,1,0,0,0,0,0,2,2.4,-4
6,Bruce Brown,2021-01-03,24-141,BRK,Washington Wizards,L (-1),Did Not Play,Did Not Play,Did Not Play,Did Not Play,...,Did Not Play,Did Not Play,Did Not Play,Did Not Play,Did Not Play,Did Not Play,Did Not Play,Did Not Play,Did Not Play,Did Not Play
7,Bruce Brown,2021-01-05,24-143,BRK,Utah Jazz,W (+34),1,23:07,3,5,...,6,6,5,0,0,1,1,6,7.6,+20
8,Bruce Brown,2021-01-07,24-145,BRK,Philadelphia 76ers,W (+13),1,17:45,2,4,...,1,1,0,1,0,2,2,5,1.1,+5
9,Bruce Brown,2021-01-08,24-146,BRK,Memphis Grizzlies,L (-5),1,22:32,1,3,...,6,6,0,0,1,3,2,2,-1.0,+1


In [17]:
'''
Need to address "Did Not Play", "Did Not Dress", and "Inactive" labels
May need to add a binary feature to indicate whether active (1), or inactive (0)
'''

'\nNeed to address "Did Not Play", "Did Not Dress", and "Inactive" labels\nMay need to add a binary feature to indicate whether active (1), or inactive (0)\n'